In [4]:
#!/usr/bin/python
# -*- coding: utf-8  -*-
#
# Reza(User:reza1615), 2019
# Distributed under the terms of MIT License (MIT)
#
import sys
sys.path.insert(0, r"G:\Outwork\wikipedia\pycore")

from pywikibot import config
from ip2geotools.databases.noncommercial import DbIpCity
from diff_match_patch import diff_match_patch
import re
import json
import pywikibot
import MySQLdb
import pandas
from bs4 import BeautifulSoup
from datetime import date
fasite = pywikibot.Site('fa', 'wikipedia')
site = pywikibot.Site('fa', 'wikipedia')
_cache = {}

In [4]:
def revision_info (revid):
    if _cache.get(tuple([revid, fasite, 'Revision_info'])):
        return _cache[tuple([revid, fasite, 'Revision_info'])]
    #https://fa.wikipedia.org/w/api.php?action=query&prop=revisions&revids=347819&rvprop=user|ids|tags|comment|timestamp|size
    params = {
        'action': 'query',
        'prop': 'revisions',
        'revids': revid,
        'rvprop': 'user|ids|tags|comment|timestamp|size'
    }
    query = pywikibot.data.api.Request(site=fasite, **params).submit()
    pagedict=query[u'query'][u'pages']
    pagedict=pagedict[list(pagedict.keys())[0]]
    page_id=pagedict['pageid']
    page_title=pagedict["title"]
    user=pagedict["revisions"][0]['user']
    timestamp=pagedict["revisions"][0]['timestamp']
    info_dict={revid:{'title':page_title,'id':page_id,'user':user,'time':timestamp}}
    
    _cache[tuple([revid, fasite, 'Revision_info'])]=info_dict
    return info_dict

def compare_texts (text1,text2):
    dmp = diff_match_patch()
    patches = dmp.patch_make(text1,text2)
    diff = dmp.patch_toText(patches)
    return diff

def compare_revisions(id1,id2):
    #https://fa.wikipedia.org/w/api.php?action=compare&fromrev=24275599&torev=26278079
    params = {
        'action': 'compare',
        'fromrev': id1,
        'torev': id2,
    }
    query = pywikibot.data.api.Request(site=fasite, **params).submit()
    Compare_txt=query[u'compare'][u'*']
    soup = BeautifulSoup(Compare_txt)
    added_list = soup.findAll("td", {"class": "diff-addedline"})
    deleted_list = soup.findAll("td", {"class": "diff-deletedline"})

    return added_list,deleted_list

def get_user_R_G (UserName):
    if _cache.get(tuple([UserName, fasite, 'user'])):
        return _cache[tuple([UserName, fasite, 'user'])]
    UserName = UserName.replace(u' ', u'_')
    #https://fa.wikipedia.org/w/api.php?action=query&list=users&ususers=Ahmad252&usprop=groups|rights|editcount|registration
    params = {
        'action': 'query',
        'list': 'users',
        'ususers': UserName,
        'usprop': 'groups|rights|editcount|registration',
    }
    query = pywikibot.data.api.Request(site=fasite, **params).submit()
    user_rights=query[u'query'][u'users'][0]["rights"]
    user_Groups=query[u'query'][u'users'][0]["groups"]
    user_editcount=query[u'query'][u'users'][0]["editcount"]
    user_registration_year=query[u'query'][u'users'][0]["registration"].split('-')[0]
    
    _cache[tuple([UserName, fasite, 'user'])] = user_rights,user_Groups,user_editcount,user_registration_year
    return user_rights,user_Groups,user_editcount,user_registration_year

def Page_veiw (title):
    if _cache.get(tuple([title, fasite, 'Page_veiw'])):
        return _cache[tuple([title, fasite, 'Page_veiw'])]
    title = title.replace(u' ', u'_')
    #https://fa.wikipedia.org/w/api.php?action=query&titles=%D8%A7%DB%8C%D8%B1%D8%A7%D9%86&prop=pageviews
    params = {
        'action': 'query',
        'titles': 'title',
        'prop': 'pageviews',
    }
    query = pywikibot.data.api.Request(site=fasite, **params).submit()
    dict=query[u'query'][u'pages']
    page_views=dict[list(dict.keys())[0]]["pageviews"]
    Page_last_month_view=sum([page_views[i] for i in page_views][-30:])
    _cache[tuple([title, fasite, 'Page_veiw'])]=Page_last_month_view
    return Page_last_month_view

def watchers (title):
    if _cache.get(tuple([title, fasite, 'watchers'])):
        return _cache[tuple([title, fasite, 'watchers'])]
    title = title.replace(u' ', u'_')
    #https://fa.wikipedia.org/w/api.php?action=query&prop=info&inprop=visitingwatchers&titles=%D8%A7%DB%8C%D8%B1%D8%A7%D9%86
    params = {
        'action': 'query',
        'titles': title,
        'prop': 'info',
        'inprop': 'watchers|visitingwatchers'
    }
    query = pywikibot.data.api.Request(site=fasite, **params).submit()
    dict=query[u'query'][u'pages']
    visitingwatchers=dict[list(dict.keys())[0]]["visitingwatchers"]
    watchers=dict[list(dict.keys())[0]]["watchers"]
    _cache[tuple([title, fasite, 'Page_veiw'])] = visitingwatchers,watchers
    return visitingwatchers,watchers

def revsion_txt (title, rvstart):
    title = title.replace(u' ', u'_')
    #https://fa.wikipedia.org/w/api.php?action=query&prop=revisions&titles=چنگال&rvlimit=10&rvslots=main&rvprop=content&rvdir=newer&rvstart=2016-07-01T00:00:00Z&rvexcludeuser=SSethiایران
    params = {
        'action': 'query',
        'titles': title,
        'prop': 'revisions',
        'rvlimit':1,
        'rvslots':'main',
        'rvprop':'content|flags|ids|timestamp|user|comment|tags|size',
        'rvdir':'newer',
        'rvstart':rvstart
    }
    query = pywikibot.data.api.Request(site=fasite, **params).submit()
    revision=query[u'query'][u'pages'][list(dict.keys())[0]]["pageid"]['revisions'][0]
    flags=revision ["flags"]
    revids=revision ["revid"]
    users=revision ["user"]
    timestamp=revision ["timestamp"]
    size=revision ["size"]
    comment=revision ["comment"]
    tags=revision ["tags"]
    txt=revision ["content"]
    return [flags,revids,users,timestamp,size,comment,tags ,txt]

def history_log (title,year,RevID):
    title = title.replace(u' ', u'_')
    #https://fa.wikipedia.org/w/api.php?action=query&prop=revisions&titles=چنگال&rvlimit=10&rvslots=main&rvprop=timestamp|user|comment|tags|size|ids&rvdir=newer&rvstart=2016-07-01T00:00:00Z&rvexcludeuser=SSethiایران
    params = {
        'action': 'query',
        'titles': title,
        'prop': 'revisions',
        'rvlimit':500,
        'rvslots':'main',
        'rvprop':'flags|ids|timestamp|user|comment|tags|size',
        'rvdir':'newer',
        'rvstart':str(year-3)+'-01-01T00:00:00Z',
        'rvexcludeuser':'Ali'
    }
    query = pywikibot.data.api.Request(site=fasite, **params).submit()
    dict=query[u'query'][u'pages']
    dict=dict[list(dict.keys())[0]]
    pageid=dict["pageid"]
    revisions=dict["pageid"]['revisions']
    flags,revids,users,timestamp,size,comment,tags,current=[],[],[],[],[],[],[],[]
    for i in revisions:
        if RevID == i["revid"]:
            current = revsion_txt (title, i["timestamp"])
        flags.append(i["flags"])
        revids.append(i["revid"])
        users.append(i["user"])
        timestamp.append(i["timestamp"])
        size.append(i["size"])
        comment.append(i["comment"])
        tags.append(i["tags"])
    return pageid,flags,revids,users,timestamp,size,comment,tags ,current

def get_ip_city(IP):
    if _cache.get(tuple([IP, fasite, 'ip_city'])):
        return _cache[tuple([IP, fasite, 'ip_city'])]
    #ip='147.229.2.90'
    response = DbIpCity.get(IP, api_key='free')
    response=json.loads(response.to_json())
    #'{"ip_address": "5.121.94.147", "city": "Tehr\\u0101n", "region": "Tehr\\u0101n", "country": "IR", "latitude": 35.7006177, "longitude": 51.4013785}'
    _cache[tuple([IP, fasite, 'ip_city'])] = response['city']+', '+response['country']
    return  response['city']+', '+response['country']

def sql_revert_by_user (user):
    user = user.replace(u' ', u'_')
    if _cache.get(tuple([user, fasite, 'revert_by_user'])):
        return _cache[tuple([user, fasite, 'revert_by_user'])]
        
    query = """SELECT count (*)
                FROM (select rev_id,rev_page,rev_actor,rev_comment_id from revision where rev_timestamp > NOW() - INTERVAL 6 MONTH) R 
                JOIN page ON R.rev_page=page_id AND page_namespace = 0
                JOIN actor ON R.rev_actor = actor_id and actor_name LIKE '"""+user +"""'
                JOIN comment ON R.rev_comment_id = comment_id and   
                comment_text LIKE '%خنثی‌سازی%' and comment_text NOT LIKE '%"""+user +"""%'
                group by R.rev_id"""
                
    cn = MySQLdb.connect("fawiki.labsdb", db=site.dbName()+ '_p', user=config.db_username, passwd=config.db_password)
    cur = cn.cursor()
    cur.execute(query)
    result = cur.fetchall()
    cn.close()
    _cache[tuple([user, fasite, 'revert_by_user'])] = len(result)
    return len(result)

def sql_revert_by_other (user):
    user = user.replace(u' ', u'_')
    if _cache.get(tuple([user, fasite, 'revert_by_other'])):
        return _cache[tuple([user, fasite, 'revert_by_other'])]
        
    query = """SELECT count (*)
                FROM (select rev_id,rev_page,rev_actor,rev_comment_id from revision where rev_timestamp > NOW() - INTERVAL 6 MONTH) R 
                JOIN page ON R.rev_page=page_id AND page_namespace = 0
                JOIN actor ON R.rev_actor = actor_id and actor_name NOT LIKE '"""+user +"""'
                JOIN comment ON R.rev_comment_id = comment_id and   
                comment_text LIKE '%خنثی‌سازی%' and comment_text LIKE '%"""+user +"""%'
                group by R.rev_id"""
    cn = MySQLdb.connect("fawiki.labsdb", db=site.dbName()+ '_p', user=config.db_username, passwd=config.db_password)
    cur = cn.cursor()
    cur.execute(query)
    result = cur.fetchall()
    cn.close()
    _cache[tuple([user, fasite, 'revert_by_other'])] = len(result)
    return len(result)

def sql_blocked_users ():
    query = """ SELECT log_title, COUNT(*)
                FROM logging
                WHERE log_type = "block"
                    AND log_timestamp > NOW() - INTERVAL 1 YEAR
                    AND REGEXP_REPLACE(log_title, '[0-9\.]+', '') <> ""
                GROUP BY log_title
                ORDER BY COUNT(*) DESC;"""
    cn = MySQLdb.connect("fawiki.labsdb", db=site.dbName()+ '_p', user=config.db_username, passwd=config.db_password)
    cur = cn.cursor()
    cur.execute(query)
    BlockedUsers = cur.fetchall()
    cn.close()
    return BlockedUsers

In [39]:
def get_links():
    project_page='ویکی‌پدیا:واحد ضد خرابکاری/ربات/ارزشیابی'
    extenstions=['','/بایگانی ۲','/بایگانی ۱']
    fa_text=''
    for extend in extenstions:
        page_link=project_page+extend
        fapage = pywikibot.Page(fasite, page_link)
        fa_text += fapage.get().strip()+'\n'
    return fa_text.replace('\r','').split('\n')

def filter_links(links):
    link_dict={}
    for link in links:
        if not '#' in link or not '|' in link:
            continue
        diff_id=link.split('|')[1]
        tag=link.split('|')[2].strip().split('-')
        #user=link.split('|')[3]
        if diff_id in link_dict:
            link_dict[diff_id]=list(set(link_dict[diff_id]+tag))
        else:
            link_dict[diff_id]=tag
    return link_dict

def link_analyzer (link):#---------------
    RevID= link.split()
    id2= link.split()
    UserName= link.split()
    title= link.split()
    return RevID,id2,UserName,title

def Mixer ():
    # Mix to each other
    # meta data
    pd.to_csv('MetaData.csv') #-------- append
    
def run (rev_id):
    year=date.today().year
    year='2019' # should be today ()
    rev_dict = revision_info (rev_id)
    #     {26790691: {
    #                 'title': 'رونالدینیو',
    #                    'id': 6978,
    #                  'user': 'Mmehdih',
    #                  'time': '2019-08-14T12:01:13Z'
    #                }
    #     }
    title = rev_dict[rev_id]['title']
    user = rev_dict[rev_id]['user']
    time = rev_dict[rev_id]['time']
    print('get rev_info ....')
    print(title,user,time)
    
    #     diff_dict = compare_texts (text1,text2)
    #     Compare_txt = compare_revisions(RevID,id2)
    
   
    #Page_last_month_view = Page_veiw (title)
    #print('get Page_veiw ....')
    #print(Page_last_month_view)
    
#     visitingwatchers, thewatchers = watchers (title)
#     print('get watchers ....')
#     print(visitingwatchers, thewatchers)
    
#     pageid,flags,revids,users,timestamp,size,comment,tags ,current = history_log (title,year,rev_id)
#     print('get history_log ....')
#     print(pageid,flags,revids,users,timestamp,size,comment,tags ,current)
    
#     BlockedUsers_list = sql_blocked_users ()
    
    # IP and users
    if not re.sub('[0-9\.]+','',user).strip():
        the_city= get_ip_city(user)
        print('get get_ip_city ....')
        print(the_city)
    else:
        user_rights,user_Groups,user_editcount,user_registration_year = get_user_R_G (user)
        print('get get_user_R_G ....')
        print(user_rights,user_Groups,user_editcount,user_registration_year)
#         len_revert_by= sql_revert_by_user (user)
#         print('get sql_revert_by_user ....')
#         print(len_revert_by)
#         len_reverted= sql_revert_by_other (user)
#         print('get sql_revert_by_other ....')
#         print(len_reverted)
        pass
        
    # Mixer
    #Mixer ()
    
def main():
    links = get_links()
    link_dict=filter_links(links)
    for link in link_dict:
        run (link)
        #break
        

if __name__ == '__main__':
    main()

get rev_info ....
دریاچه ارومیه ArefKabi 2019-07-19T21:03:46Z
get get_user_R_G ....
['autopatrol', 'move-categorypages', 'abusefilter-log', 'abusefilter-view', 'abusefilter-log-detail', 'autoreview', 'block', 'delete', 'deleterevision', 'mergehistory', 'protect', 'suppressredirect', 'deletedtext', 'deletedhistory', 'extendedconfirmed', 'patrol', 'rollback', 'upload', 'reupload', 'reupload-own', 'autoerview', 'autoreviewrestore', 'validate', 'review', 'oathauth-enable', 'createaccount', 'read', 'edit', 'createtalk', 'writeapi', 'viewmywatchlist', 'editmywatchlist', 'viewmyprivateinfo', 'editmyprivateinfo', 'editmyoptions', 'urlshortener-create-url', 'centralauth-merge', 'vipsscaler-test', 'flow-hide', 'move-rootuserpages', 'createpage', 'minoredit', 'editmyusercss', 'editmyuserjson', 'editmyuserjs', 'purge', 'sendemail', 'applychangetags', 'changetags', 'spamblacklistlog', 'flow-lock', 'mwoauthmanagemygrants', 'move', 'collectionsaveasuserpage', 'collectionsaveascommunitypage', 'autocon

  # This is added back by InteractiveShellApp.init_path()



get rev_info ....
دریاچه ارومیه Arash.pt 2019-07-19T20:40:38Z
get get_user_R_G ....
['extendedconfirmed', 'suppressredirect', 'noratelimit', 'deleterevision', 'deletelogentry', 'editcontentmodel', 'block', 'createaccount', 'delete', 'deletedhistory', 'deletedtext', 'undelete', 'editinterface', 'editsitejson', 'edituserjson', 'import', 'move', 'move-subpages', 'move-rootuserpages', 'move-categorypages', 'patrol', 'autopatrol', 'protect', 'editprotected', 'rollback', 'upload', 'reupload', 'reupload-shared', 'unwatchedpages', 'autoconfirmed', 'editsemiprotected', 'ipblock-exempt', 'blockemail', 'markbotedits', 'apihighlimits', 'browsearchive', 'movefile', 'mergehistory', 'managechangetags', 'deletechangetags', 'abusefilter-revert', 'abusefilter-log', 'abusefilter-log-private', 'abusefilter-view', 'oathauth-enable', 'autoreview', 'stablesettings', 'movestable', 'tboverride', 'titleblacklistlog', 'transcode-reset', 'transcode-status', 'urlshortener-create-url', 'globalblock-whitelist', 'nuk

get rev_info ....
دانشگاه فنی خاورمیانه Shahnamk 2019-07-22T19:43:19Z
get get_user_R_G ....
['autopatrol', 'move-categorypages', 'abusefilter-log', 'abusefilter-view', 'abusefilter-log-detail', 'autoreview', 'extendedconfirmed', 'patrol', 'review', 'validate', 'rollback', 'autoreviewrestore', 'createaccount', 'read', 'edit', 'createtalk', 'writeapi', 'viewmywatchlist', 'editmywatchlist', 'viewmyprivateinfo', 'editmyprivateinfo', 'editmyoptions', 'urlshortener-create-url', 'centralauth-merge', 'vipsscaler-test', 'flow-hide', 'reupload-own', 'move-rootuserpages', 'createpage', 'minoredit', 'editmyusercss', 'editmyuserjson', 'editmyuserjs', 'purge', 'sendemail', 'applychangetags', 'changetags', 'spamblacklistlog', 'flow-lock', 'mwoauthmanagemygrants', 'reupload', 'upload', 'move', 'collectionsaveasuserpage', 'collectionsaveascommunitypage', 'autoconfirmed', 'editsemiprotected', 'skipcaptcha', 'flow-edit-post', 'transcode-reset', 'movestable'] ['autopatrolled', 'extendedconfirmed', 'patrol

KeyboardInterrupt: 

In [6]:
links = get_links()
link_dict=filter_links(links)

In [ ]:
#link_dict={'26634694': ['Nحذف_نامناسب', 'Nواگردانی']}

In [3]:
revision_info (26790691)

  # This is added back by InteractiveShellApp.init_path()



{26790691: {'title': 'رونالدینیو',
  'id': 6978,
  'user': 'Mmehdih',
  'time': '2019-08-14T12:01:13Z'}}

In [41]:
old=26790691
new=26798425
text,b= compare_revisions(old,new)

2019

In [42]:
soup = BeautifulSoup(text)
added_list = soup.findAll("td", {"class": "diff-addedline"})
deleted_list = soup.findAll("td", {"class": "diff-deletedline"})
#print (soup.prettify())

TypeError: expected string or bytes-like object

In [ ]:
text


In [ ]:
b


In [ ]:
print (soup.prettify())

In [47]:
def revsion_txt (rvstart):
    #https://fa.wikipedia.org/w/api.php?action=query&prop=revisions&titles=چنگال&rvlimit=10&rvslots=main&rvprop=content&rvdir=newer&rvstart=2016-07-01T00:00:00Z&rvexcludeuser=SSethiایران
    params = {
        'action': 'query',
        'prop': 'revisions',
        'rvlimit':1,
        'rvslots':'main',
        'rvprop':'content|flags|ids|timestamp|user|comment|tags|size',
        'rvdir':'newer',
        'rvstart':rvstart
    }
    query = pywikibot.data.api.Request(site=fasite, **params).submit()
    revision=query[u'query'][u'pages'][list(dict.keys())[0]]["pageid"]['revisions'][0]
    flags=revision ["flags"]
    revids=revision ["revid"]
    users=revision ["user"]
    timestamp=revision ["timestamp"]
    size=revision ["size"]
    comment=revision ["comment"]
    tags=revision ["tags"]
    txt=revision ["content"]
    return [flags,revids,users,timestamp,size,comment,tags ,txt]

In [48]:
revsion_txt (26790691)

  if sys.path[0] == '':



KeyError: 'pages'